In [32]:
import pandas as pd
import numpy as np
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate  
from langchain_community.document_loaders import PyPDFDirectoryLoader  
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [11]:
load_dotenv()
os.environ["GROQ_API_KEY"]  = os.getenv("GROQ_API_KEY")


In [15]:
llm = ChatGroq(model="qwen-2.5-32b")
llm.invoke("Hi How are you ?")

AIMessage(content="Hello! As an AI, I don't have feelings or physical sensations, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 34, 'total_tokens': 67, 'completion_time': 0.165, 'prompt_time': 0.003911969, 'queue_time': 0.049031211, 'total_time': 0.168911969}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ec482e1-c66e-46da-9aeb-83b36ff6fc06-0', usage_metadata={'input_tokens': 34, 'output_tokens': 33, 'total_tokens': 67})

In [18]:
prompt = ChatPromptTemplate.from_template("""
Assume you are an Assistant of the person in the context.
Please provide the most accurate answer based on the questions in formal and attractive way. 
Answer the questions based on the context only
<context>
{context}
<context>
Question: {input}
""")

In [31]:
def create_vector_embeddings():
    if 'vector' not in st.session_state:
        st.session_state.loader = PyPDFDirectoryLoader("docs")
        st.session_state.docs = st.session_state.loader.load()
        st.session_state.text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
        st.session_state.final_docs = st.session_state.text_splitter.split_documents(st.session_state.docs)
        st.session_state.embeddings = OllamaEmbeddings(model="llama3.2:1b")
        st.session_state.vectorstore = FAISS.from_documents(documents=st.session_state.final_docs, embedding=st.session_state.embeddings)
        

In [ ]:
st.set_page_config(page_title="Know About Ananthmanoj")
st.title("Know About Ananthmanoj")


try:
    with st.spinner("Getting ready..."):
        create_vector_embeddings()
        st.success("Connected")
    
    user_prompt = st.text_input("Hi, I am ___, What quetion do you have about my Master ?")
    if user_prompt:
        with st.spinner("He has many good things...let me summarise",show_time= True):
            doc_chain = create_stuff_documents_chain(llm=llm, prompt= prompt)
            retriever = st.session_state.vectorstore.as_retriever()
            rertriever_chain = create_retrieval_chain(retriever, doc_chain)
            response = rertriever_chain.invoke({"input":user_prompt})
            st.success(response['answer'])

        with st.expander("Similar results :"):
            for i,d in enumerate(response["context"]):
                st.write(d.page_content)
                st.write("------------------------------------")

    
except Exception as e:
    st.write(f"ERROR!! Contact My Master \n error: ({e})")